# 3. RGSW Operation

Now we have all the required building blocks for RGSW operations.
We already checked RLWE' multiplication is working.

Call all previous methods

In [526]:
# Functions from previous lecturenote
import torch
import math

stddev = 3.2
logQ = 27

N = 2**10
Q = 2**logQ


def keygen(dim):
    return torch.randint(2, size = (dim,))

def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)

def errgen(stddev, N):
    e = torch.round(stddev*torch.randn(N))
    e = e.squeeze()
    return e.to(torch.int)

def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

def polymult(a, b, dim, modulus):
    res = torch.zeros(dim).to(torch.int)
    for i in range(dim):
        for j in range(dim):
            if i >= j:
                res[i] += a[j]*b[i-j]
                res[i] %= modulus
            else:
                res[i] -= a[j]*b[i-j] # Q - x mod Q = -x
                res[i] %= modulus

    res %= modulus
    return res

root_powers = torch.arange(N//2).to(torch.complex128)
root_powers = torch.exp((1j*math.pi/N)*root_powers)

root_powers_inv = torch.arange(0,-N//2,-1).to(torch.complex128)
root_powers_inv = torch.exp((1j*math.pi/N)*root_powers_inv)

def negacyclic_fft(a, N, Q):
    acomplex = a.to(torch.complex128)

    a_precomp = (acomplex[...,:N//2] + 1j * acomplex[..., N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((b.real, b.imag), dim=-1)

    aint = a.to(torch.int32)
    # only when Q is a power-of-two
    aint &= Q-1

    return aint

In [527]:
# make an RLWE encryption of message
def encrypt_to_fft(m, sfft):
    ct = torch.stack([errgen(stddev, N), uniform(N, Q)])
    ctfft = negacyclic_fft(ct, N, Q)

    ctfft[0] += -ctfft[1]*sfft + negacyclic_fft(m, N, Q)

    return ctfft

def normalize(v, logQ):
    # same as follows but no branch
    """
    if v > Q//2:
        v -= Q
    """
    # vmod Q when Q is a power-of-two
    Q = (1 << logQ)
    v &= Q-1
    # get msb
    msb = (v & Q//2) >> (logQ - 1)
    v -= (Q) * msb
    return v

def decrypt_from_fft(ctfft, sfft):
    return normalize(negacyclic_ifft(ctfft[0] + ctfft[1]*sfft, N, Q), logQ)

In [528]:
# we will use B-ary decomposition, i.e., cut digits by logB bits
d = 3
logB = 7

decomp_shift = logQ - logB*torch.arange(d,0,-1).view(d,1)
mask = (1 << logB) - 1

gvector = 1 << decomp_shift

In [529]:
def decompose(a):
    
    assert len(a.size()) <= 2
    # for RLWE'
    if len(a.size()) == 1:
        res = (a.unsqueeze(0) >> decomp_shift.view(d, 1)) & mask
        return res
    # for RGSW
    elif len(a.size()) == 2:
        res = (a.unsqueeze(0) >> decomp_shift.view(d, 1, 1)) & mask
        return res

msbmask = 0
for i in decomp_shift:
    msbmask += (1<<(i+logB-1))

bin(msbmask)[2:]

# about twice heavier than unsigned decomposition
# it returns value -B/2 <= * <= B/2, not < B/2, but okay
def signed_decompose(a):
    # carry
    da = decompose(a + (a & msbmask))
    # -B
    da -= decompose((a & msbmask))
    return da

In [530]:
def encrypt_rlwep_fft(z, skfft):
    # it has a dimension of d, 2, N
    rlwep = torch.zeros(d, 2, N, dtype=torch.int32)

    # generate the 'a' part
    rlwep[:, 1, :] = torch.randint(Q, size = (d, N), dtype= torch.int32)

    # add error on b
    # INSECURE: need to be fixed later
    rlwep[:, 0, :] = torch.round(stddev * torch.randn(size = (d, N)))

    # following is equal to rlwep %= Q, but a faster version
    rlwep &= (Q-1)
    rlwep = normalize(rlwep, logQ)

    # do fft for easy a*s
    rlwepfft = negacyclic_fft(rlwep, N, Q)

    # now b = -a*sk + e
    rlwepfft[:, 0, :] -= rlwepfft[:, 1, :] * skfft.view(1,N//2)

    # add decomposition of z * vec{g}
    gz = gvector * z
    rlwepfft[:, 0, :] += negacyclic_fft(gz, N, Q)

    return rlwepfft

## 3.1. Test the RLWE' encryptions

1. Key gen and encryption

In [531]:
# generate two keys
s1 = keygen(N)
s2 = keygen(N)

s1fft = negacyclic_fft(s1, N, Q)
s2fft = negacyclic_fft(s2, N, Q)

In [532]:
m = torch.zeros((N), dtype=torch.int32)
m[2] = Q//4
m[3] = Q//4
m[5] = Q//4

m[:10]

tensor([       0,        0, 33554432, 33554432,        0, 33554432,        0,
               0,        0,        0], dtype=torch.int32)

In [533]:
ctfft = encrypt_to_fft(m, s1fft)
mdec = decrypt_from_fft(ctfft, s1fft)
mdec[:10]

tensor([       1,        0, 33554434, 33554434,       -2, 33554431,       -8,
              -5,        2,        1], dtype=torch.int32)

2. Key switching key ($s_1 \rightarrow s_2$) gen

In [534]:
kskfft = encrypt_rlwep_fft(s1, s2fft)

In [535]:
kskfft.size()

torch.Size([3, 2, 512])

3. Try key switching

In [536]:
def keyswitch(ctfft, kskfft):
    ct = negacyclic_ifft(ctfft, N, Q)
    
    da = signed_decompose(ct[1])
    dafft = negacyclic_fft(da, N, Q)

    # a * RLWE'(s1)
    prodfft = dafft.view(d, 1, N//2) * kskfft
    prodsumfft = torch.sum(prodfft, dim = 0)

    # (b,0) + a*RLWE'(s1)
    prodsumfft[0] += ctfft[0]

    return prodsumfft


In [537]:
switched = keyswitch(ctfft, kskfft)

mks = decrypt_from_fft(switched, s2fft)
print("m:\n",m[:10])
print("decrypted:\n",mks[:10])
print("decrypted (scaled to 1):\n",mks[:10]/(Q//4))

m:
 tensor([       0,        0, 33554432, 33554432,        0, 33554432,        0,
               0,        0,        0], dtype=torch.int32)
decrypted:
 tensor([    6209,    23094, 33577246, 33552794,     5587, 33577680,    17151,
             444,    10883,    12415], dtype=torch.int32)
decrypted (scaled to 1):
 tensor([1.8504e-04, 6.8825e-04, 1.0007e+00, 9.9995e-01, 1.6651e-04, 1.0007e+00,
        5.1114e-04, 1.3232e-05, 3.2434e-04, 3.7000e-04])


Bonus: unsigned decomposition version

In [538]:
def unsigned_keyswitch(ctfft, kskfft):
    ct = negacyclic_ifft(ctfft, N, Q)
    
    da = decompose(ct[1])
    dafft = negacyclic_fft(da, N, Q)

    # a * RLWE'(s1)
    prodfft = dafft.view(d, 1, N//2) * kskfft
    prodsumfft = torch.sum(prodfft, dim = 0)

    # (b,0) + a*RLWE'(s1)
    prodsumfft[0] += ctfft[0]

    return prodsumfft


In [539]:
switched_unsigned = unsigned_keyswitch(ctfft, kskfft)

umks = decrypt_from_fft(switched_unsigned, s2fft)
print("m:\n",m[:10])
print("decrypted:\n",umks[:10])
print("decrypted (scaled to 1):\n",umks[:10]/(Q//4))



m:
 tensor([       0,        0, 33554432, 33554432,        0, 33554432,        0,
               0,        0,        0], dtype=torch.int32)
decrypted:
 tensor([   12392,     3012, 33568214, 33582208,    20939, 33564397,    18491,
           21333,    18755,    36702], dtype=torch.int32)
decrypted (scaled to 1):
 tensor([3.6931e-04, 8.9765e-05, 1.0004e+00, 1.0008e+00, 6.2403e-04, 1.0003e+00,
        5.5107e-04, 6.3577e-04, 5.5894e-04, 1.0938e-03])


In [540]:
print("signed")
print("Error variance: \n", torch.var((m-mks)/Q))

print("unsigned")
print("Error variance: \n", torch.var((m-umks)/Q))


signed
Error variance: 
 tensor(7.1657e-09)
unsigned
Error variance: 
 tensor(5.8520e-09)


## 3.2. RGSW Encryption

RGSW(m) is composed of two RLWE' ciphertexts, i.e., 

$$
RGSW(m) = ( RLWE'(m), RLWE'(\boldsymbol{m_1}\cdot s) )
$$

The multiplication between RLWE and RGSW is defined as follows
$$
\circledast: RLWE \times RGSW \mapsto RLWE
$$
Let $(\boldsymbol{b}, \boldsymbol{a})$ is encryption of $\boldsymbol{m_0}$
$$
(\boldsymbol{b}, \boldsymbol{a}) \circledast RGSW(m) \\
= (\boldsymbol{b}, \boldsymbol{a}) \circledast ( RLWE'(\boldsymbol{m_1}), RLWE'(\boldsymbol{m_1}\cdot s) )\\
= \boldsymbol{b} \odot RLWE'(\boldsymbol{m_1}) 
+ \boldsymbol{a} \odot RLWE'(\boldsymbol{m_1}\cdot s)\\
= RLWE(\boldsymbol{bm_1}) + RLWE(\boldsymbol{asm_1})\\
= RLWE(\boldsymbol{(b+as)m_1})\\
= RLWE(\boldsymbol{(m_0 + e)m_1}) \\
\approx RLWE(\boldsymbol{m_0 m_1})
$$

Similar to RLWE', we can define encryption function as follows.

In [ ]:
def encrypt_rgsw_fft(z, skfft):
    # it has a dimension of d, 2, 2, N
    rgsw = torch.zeros(d, 2, 2, N, dtype=torch.int32)

    # generate the 'a' part
    rlwep[:, 1, :] = torch.randint(Q, size = (d, N), dtype= torch.int32)

    # add error on b
    # INSECURE: need to be fixed later
    rlwep[:, 0, :] = torch.round(stddev * torch.randn(size = (d, N)))

    # following is equal to rlwep %= Q, but a faster version
    rlwep &= (Q-1)
    rlwep = normalize(rlwep, logQ)

    # do fft for easy a*s
    rlwepfft = negacyclic_fft(rlwep, N, Q)

    # now b = -a*sk + e
    rlwepfft[:, 0, :] -= rlwepfft[:, 1, :] * skfft.view(1,N//2)

    # add decomposition of z * vec{g}
    gz = gvector * z
    rlwepfft[:, 0, :] += negacyclic_fft(gz, N, Q)

    return rlwepfft